# ADA Project : Dunnhumby dataset, Tell me what you buy and I will tell you who you are



## Abstract
We would like to analyse the Dunnhumby dataset. Living in a time and age where every piece of our data is stored and analysed; and being active consumers ourselves, we would like to see what informations retail chains can gather and infer about us knowing only our shopping habits. As transactions over two years of several households and their basic demographic profiles are provided, we want to see if there are any links and correlations between specific demographics (e.g. marital status, income, number of children, etc) and purchase patterns. Furthermore, if time permits it, we want to see if we can create a model predicting a consumer demographic profile from their shopping. Thus, we would like to see how "easy" and how precise it actually is for retailers to infer who their customer is by what they buy and target them with specific marketing. Basically, we want to know how much of a target we actually
are.

**Research questions:** 
- What are the main shopping trends that we can identify in this data ?
- Can we relate shopping trends to specific demographic parameters ?
- Can we predict some of these demographic parameters (age, marital statute etc) with knowing the household's habbits?
- In the opposite way, can we predict household consumption behaviour with knowing its characteristics?
- What accuracy in consumption prediction can the retailer obtain from a simple profile information?

## Task 1: Clean up the data and prepare the sets we want to keep

In [1]:
%matplotlib inline
import pandas as pd

import matplotlib.pyplot as plt
from pylab import *

import os
# to print numbers in a nice way
import locale
locale.setlocale(locale.LC_ALL, '')

'LC_CTYPE=en_US.UTF-8;LC_NUMERIC=it_IT.UTF-8;LC_TIME=it_IT.UTF-8;LC_COLLATE=en_US.UTF-8;LC_MONETARY=it_IT.UTF-8;LC_MESSAGES=en_US.UTF-8;LC_PAPER=it_IT.UTF-8;LC_NAME=it_IT.UTF-8;LC_ADDRESS=it_IT.UTF-8;LC_TELEPHONE=it_IT.UTF-8;LC_MEASUREMENT=it_IT.UTF-8;LC_IDENTIFICATION=it_IT.UTF-8'

In [2]:
os.getcwd()

'/home/marco/epfl/ada/epfl-ada-2019-project-bubble582/code'

In [3]:
'''As we said in the description of our project, we are going to concentrate on 3 of the 8 tables :
- hh_demographic.csv
- transaction_data.csv
- product.csv
In this first step, we want to load the data, and prepare it for the analysis'''

#load the data
hh_demographic = pd.read_csv('../data/dunnhumby_complete_csv/hh_demographic.csv', sep = ',')

transaction_data = pd.read_csv('../data/dunnhumby_complete_csv/transaction_data.csv', sep = ',')

product = pd.read_csv('../data/dunnhumby_complete_csv/product.csv', sep = ',')

### Task 1.A: What's actually in the dataset? 
This dataset contains household level transactions over two years from a group of 2,500 households who are frequent shoppers at a retailer. It contains all of each household’s purchases, not just those from a limited number of categories. For certain households, demographic information as well as direct marketing contact history are included. We have a look at a few samples from each table: 

#### **A. Transaction data:**
Dataset of all products purchased by households during the study. Each line in the table is what could essentially be found in a store reciept. The attributes of the dataset are the following: 

- HOUSEHOLD_KEY: identifies each household, 
- BASKET_ID: identifies a purchase occasion, 
- DAY: day when transaction occured
- PRODUCT_ID: identifies each product, 
- QUANTITY: Number of products purchased during trip
- SALES_VALUE: Amount of dollars retailer recieves from sale
- STORE_ID: identifies store, 
- COUPON_MATCH_DISC: discount applied du to retailer's match of manufacturer coupon
- COUPON_DISC: discount applied due to manufacturer coupon
- RETAIL_DISC: discount applied due to retailer's loyalty card program
- TRANS_TIME: time of day when transaction occured
- WEEK_NO: week of the transaction. Ranges from 1-102. 

In [4]:
transaction_data['STORE_ID'].is_unique

False

In [5]:
transaction_data.head(4)

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.0,1631,1,0.0,0.0


**Q1: How many transactions occured during the two years?**

In [6]:
print(f"In total there were {transaction_data.count()['household_key']:n} transactions during the two years.")

In total there were 2.595.732 transactions during the two years.


**Q2: How many purchase occasions occured during the two years?** <br>
Attention : here transactions are not what we usually think of. It's like on a reciept so the number of total unique purchases is not the number of transactions but the total of unique basket_id. 

In [7]:
print(f"In total there were {len(transaction_data['BASKET_ID'].unique()):n} purchase occasions during the two years.")

In total there were 276.484 purchase occasions during the two years.


**Q3: How many households are represented in the transactions?**

In [8]:
print(f"In total {len(transaction_data['household_key'].unique()):n} households made at least one transaction during the two years.")

In total 2.500 households made at least one transaction during the two years.


#### **B. Demographic data:**
Demographic info for a certain portion of households. Contains only the data for 801 households of 2500 though. The rest could not be aquiered. The attributes of the dataset are the following: 
 
- HOUSEHOLD_KEY : identifies each household, **unique**
- AGE_DES: estimated age range
- MARITAL_STATUS_CODE: A (Married), B (Single), C (Unknown)
- INCOME_DESC : Household income
- HOMEOWNER_DESC: Homeowner, renter, etc
- HH_COMP_DEC: Household composition
- HOUSEHOLD_SIZE_DESC: Size of household up to 5+ 
- KID_CATEGORY_DESC: Number of children present up to 3+ 

In [9]:
hh_demographic['household_key'].is_unique

True

In [10]:
hh_demographic.head(4)

,AGE_DESC,MARITAL_STATUS_CODE,INCOME_DESC,HOMEOWNER_DESC,HH_COMP_DESC,HOUSEHOLD_SIZE_DESC,KID_CATEGORY_DESC,household_key
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7
2,25-34,U,25-34K,Unknown,2 Adults Kids,3,1,8
3,25-34,U,75-99K,Homeowner,2 Adults Kids,4,2,13


**Q3: How many age categories are there ? And what are they ?**


In [11]:
print(f"In total there are {len(hh_demographic['AGE_DESC'].unique()):n} age categories")
print("The different categories are:", hh_demographic['AGE_DESC'].unique())

In total there are 6 age categories
The different categories are: ['65+' '45-54' '25-34' '35-44' '19-24' '55-64']


**Q4: How many income categories are there ? And what are they ?**

In [12]:
print(f"In total there are {len(hh_demographic['INCOME_DESC'].unique()):n} income categories")
print("The different categories are:", hh_demographic['INCOME_DESC'].unique())

In total there are 12 income categories
The different categories are: ['35-49K' '50-74K' '25-34K' '75-99K' 'Under 15K' '100-124K' '15-24K'
 '125-149K' '150-174K' '250K+' '175-199K' '200-249K']


**Q5: How many homeowner categories are there ? And what are they ?**

In [13]:
print("In total there are %d homeowner categories" %len(hh_demographic['HOMEOWNER_DESC'].unique()))
print("The different categories are:", hh_demographic['HOMEOWNER_DESC'].unique())

In total there are 5 homeowner categories
The different categories are: ['Homeowner' 'Unknown' 'Renter' 'Probable Renter' 'Probable Owner']


**Q7: How many household composition categories are there ? And what are they ?**

In [14]:
print(f"In total there are {len(hh_demographic['HH_COMP_DESC'].unique()):n} household composition categories")
print("The different categories are:", hh_demographic['HH_COMP_DESC'].unique())

In total there are 6 household composition categories
The different categories are: ['2 Adults No Kids' '2 Adults Kids' 'Single Female' 'Unknown'
 'Single Male' '1 Adult Kids']


**Q8: How many household size categories are there ? And what are they ?**

In [15]:
print(f"In total there are {len(hh_demographic['HOUSEHOLD_SIZE_DESC'].unique()):n} household size categories")
print("The different categories are:", hh_demographic['HOUSEHOLD_SIZE_DESC'].unique())

In total there are 5 household size categories
The different categories are: ['2' '3' '4' '1' '5+']


**Q9: How many kid number categories are there ? And what are they ?**

In [16]:
print(f"In total there are {len(hh_demographic['KID_CATEGORY_DESC'].unique()):n} kid number categories")
print("The different categories are:", hh_demographic['KID_CATEGORY_DESC'].unique())

In total there are 4 kid number categories
The different categories are: ['None/Unknown' '1' '2' '3+']


**Q10: Do household composition, household size, kid category match with each other?**

In [17]:
for hh_composition in hh_demographic['HH_COMP_DESC'].unique():
    print("Looking at the household composition:", hh_composition)
    hh_demographic_current_composition = hh_demographic[hh_demographic['HH_COMP_DESC'] == hh_composition]
    print("Household size unique information:", hh_demographic_current_composition['HOUSEHOLD_SIZE_DESC'].unique())
    print("Kid number unique information:", hh_demographic_current_composition['KID_CATEGORY_DESC'].unique())
    print()

Looking at the household composition: 2 Adults No Kids
Household size unique information: ['2']
Kid number unique information: ['None/Unknown']

Looking at the household composition: 2 Adults Kids
Household size unique information: ['3' '4' '5+']
Kid number unique information: ['1' '2' '3+']

Looking at the household composition: Single Female
Household size unique information: ['1' '2']
Kid number unique information: ['None/Unknown']

Looking at the household composition: Unknown
Household size unique information: ['1' '3' '2' '5+']
Kid number unique information: ['None/Unknown' '1' '3+']

Looking at the household composition: Single Male
Household size unique information: ['1' '2']
Kid number unique information: ['None/Unknown']

Looking at the household composition: 1 Adult Kids
Household size unique information: ['2' '3' '5+' '4']
Kid number unique information: ['1' '2' '3+']



The household category 2 Adults No Kids makes sense for each entry.
The categories Single Female and Single Male have come inconsistencies along the dataset. Both of them should have the household size to 1, but there are some entries with size to 2. Why? <br>
Let's now investigate the three remaining categories. For example for 2 Adults Kids we expect the household size to be 3 if the number of kids is 1, etc.

In [18]:
hh_composition = "2 Adults Kids"
hh_demographic_current_composition = hh_demographic[hh_demographic['HH_COMP_DESC'] == hh_composition]
for household_size in hh_demographic_current_composition['HOUSEHOLD_SIZE_DESC'].unique():
    tmp = hh_demographic_current_composition[hh_demographic_current_composition['HOUSEHOLD_SIZE_DESC'] == household_size]
    print(f"For house hold size {household_size}, with 2 adults, there are", tmp['KID_CATEGORY_DESC'].unique(), "kid categories")
print()
hh_composition = "1 Adult Kids"
hh_demographic_current_composition = hh_demographic[hh_demographic['HH_COMP_DESC'] == hh_composition]
for household_size in hh_demographic_current_composition['HOUSEHOLD_SIZE_DESC'].unique():
    tmp = hh_demographic_current_composition[hh_demographic_current_composition['HOUSEHOLD_SIZE_DESC'] == household_size]
    print(f"For house hold size {household_size}, with 1 adult, there are", tmp['KID_CATEGORY_DESC'].unique(), "kid categories")

For house hold size 3, with 2 adults, there are ['1'] kid categories
For house hold size 4, with 2 adults, there are ['2'] kid categories
For house hold size 5+, with 2 adults, there are ['3+'] kid categories

For house hold size 2, with 1 adult, there are ['1'] kid categories
For house hold size 3, with 1 adult, there are ['2' '1'] kid categories
For house hold size 5+, with 1 adult, there are ['3+'] kid categories
For house hold size 4, with 1 adult, there are ['2' '3+'] kid categories


We can see that there are some inconsistencies. We fix them in the notebook fixing_household_composition.ipynb

**Q11: How many marital status categories are there? And what are they?**

In [19]:
print("In total there are %d marital status categories" %len(hh_demographic['MARITAL_STATUS_CODE'].unique()))
print("The different categories are:", hh_demographic['MARITAL_STATUS_CODE'].unique())

In total there are 3 marital status categories
The different categories are: ['A' 'U' 'B']


For the marital status, the categories are not obvious:
- 'A' = 'married'
- 'B' = 'Single'
- 'U' = 'Unknown'

**Q11: How many households are there?**

In [20]:
print(f"In total there are {hh_demographic.count()['household_key']:n} households for which we have the demographic data.")

In total there are 801 households for which we have the demographic data.


**Note for the bubble group :**

**Should we keep in the transaction data only the households for which we have the demographic data?? could be interesting considering the fact that we want to get insights on the shopping behavior according to the demographic data**

#### **C. Product data:**
Information on each product sold such as type of product, national or private label and a brand identifier. The attributes of the dataset are the following: 
- PRODUCT_ID: **unique**, identifies product
- DEPARMENT: groups similar products together
- COMMODITY_DESC: groups similar products together at a lower level
- SUB_COMMODITY_DESC: groups similar products together at the lowest level. 
- MANUFACTURER: code that links products with the same manufacturer together 
- BRAND: indicates private or national label brand
- CURR_SIZE_OF_PRODUCT: indicates package size (not available for all) 

Let's have a look: 

In [ ]:
product.head(4)

**Q10: How many products are there ?**

In [ ]:
# Are the products IDs unique ?
product['PRODUCT_ID'].is_unique

In [ ]:
print("In total there are "+ f"{product.count()['PRODUCT_ID']:,d}" +" products")

**Q11: How many department categories are there ? And what are they ?**

In [ ]:
print("In total there are "+ f"{len(product['DEPARTMENT'].unique()) :,d}"+ " department categories" )
print("The different categories are:", product['DEPARTMENT'].unique())

**Q12: Are all produts in the product dataset represented in transactions ?**
There are 92 353 products. As for the households, we can investigate whether all the products are represented in the *transaction_data* table.

In [ ]:
print("There are "+ f"{len(transaction_data['PRODUCT_ID'].unique()):,d}" +" products in the transactions table" )

There are 92 339 products represented in the *transaction_data* table, meaning that only **14** are not represented. We can more easily imagine to do an inner join, and just drop those 14 products. 

**Q13: Which are these 14 products that are never sold ?**

### TASK 1.B: Simple plots

#### A. HH-demographic

In [ ]:
hh_demographic.head(4)

For now, the categories in this data frame are not arranged in a meaninful way, meaning that if we would make some plots now, we would not have the age categories ranged in ascending or descending order for example. 
Thus, we first want to arrange them, before making some exploratory plots.

In [ ]:
ordered_age= ['19-24','25-34','35-44','45-54','55-64', '65+' ]

hh_demographic['AGE_DESC'] = pd.Categorical(hh_demographic['AGE_DESC'],
                      ordered = True,
                      categories = ordered_age)

print ('The order of the age categories is :', ordered_age)

In [ ]:
ordered_income= ['Under 15K','15-24K','25-34K','35-49K','50-74K','75-99K','100-124K',
                 '125-149K','150-174K','175-199K','200-249K','250K+']

hh_demographic['INCOME_DESC'] = pd.Categorical(hh_demographic['INCOME_DESC'],
                      ordered = True,
                      categories = ordered_income)

print ('The order of the income categories is :', ordered_income)

In [ ]:
ordered_homeowner= ['Unknown','Probable Renter','Renter','Probable Owner','Homeowner']

hh_demographic['HOMEOWNER_DESC'] = pd.Categorical(hh_demographic['HOMEOWNER_DESC'],
                      ordered = True,
                      categories = ordered_homeowner)

print ('The order of the homeowner categories is :', ordered_homeowner)

In [ ]:
ordered_hh_comp= ['Unknown','Single Female','Single Male','1 Adult Kids','2 Adults No Kids','2 Adults Kids']

hh_demographic['HH_COMP_DESC'] = pd.Categorical(hh_demographic['HH_COMP_DESC'],
                      ordered = True,
                      categories = ordered_hh_comp)

print ('The order of the household composition categories is :', ordered_hh_comp)

In [ ]:
ordered_hh_size= ['1','2','3','4','5+']

hh_demographic['HOUSEHOLD_SIZE_DESC'] = pd.Categorical(hh_demographic['HOUSEHOLD_SIZE_DESC'],
                      ordered = True,
                      categories = ordered_hh_size)

print ('The order of the household size categories is :', ordered_hh_size)

In [ ]:
ordered_kid_number= ['None/Unknown','1','2','3+']

hh_demographic['KID_CATEGORY_DESC'] = pd.Categorical(hh_demographic['KID_CATEGORY_DESC'],
                      ordered = True,
                      categories = ordered_kid_number)

print ('The order of the kid number categories is :', ordered_kid_number)

In [ ]:
ordered_marital_status= ['A','B','U']

hh_demographic['MARITAL_STATUS_CODE'] = pd.Categorical(hh_demographic['MARITAL_STATUS_CODE'],
                      ordered = True,
                      categories = ordered_marital_status)

print ('The order of the marital status categories is :', ordered_marital_status)

Now that all the categories in this data frame are ranged in a meaningful way, let's make some simple plots to have an idea of the characteristics of the population which we study.

In [ ]:
fig1 = plt.figure(figsize=(20,20))

plt.subplot(2, 2, 1)
hh_demographic['AGE_DESC'].value_counts(sort = False).plot(kind = 'bar', title = 'Age histogram')

plt.subplot(2, 2, 2)
hh_demographic['MARITAL_STATUS_CODE'].value_counts(sort = False).plot(kind='bar', title = 'marital status histogram')

plt.subplot(2,2,3)
hh_demographic['INCOME_DESC'].value_counts(sort = False).plot(kind='bar', title = 'Income Histograms')

plt.subplot(2,2,4)
hh_demographic['HOMEOWNER_DESC'].value_counts(sort = False).plot(kind='bar', title = 'Homeowner histogram')

plt.show()

In [ ]:
fig2 = plt.figure(figsize=(20,20))

plt.subplot(2,2,1)
hh_demographic['HH_COMP_DESC'].value_counts(sort = False).plot(kind='bar', title = 'Household composition histogram')

plt.subplot(2,2,2)
hh_demographic['HOUSEHOLD_SIZE_DESC'].value_counts(sort = False).plot(kind='bar', title = 'Household size histogram')

plt.subplot(2,2,3)
hh_demographic['KID_CATEGORY_DESC'].value_counts(sort = False).plot(kind='bar', title = 'Kid categories')

plt.show()

**B.Product data**<br/>
We reproduce the ordering step for the product table

In [ ]:
product.head(4)

In [ ]:
ordered_manufacturer= product.MANUFACTURER.astype(int).sort_values()
ordered_manufacturer= ordered_manufacturer.unique()

product['MANUFACTURER'] = pd.Categorical(product['MANUFACTURER'],
                      ordered = True,
                      categories = ordered_manufacturer)

print ('The order of the manufacturer categories is :', ordered_manufacturer)

In [ ]:
ordered_department= ['GROCERY','FROZEN GROCERY','GRO BAKERY','PASTRY',
        'SEAFOOD','SEAFOOD-PCKGD','MEAT', 'MEAT-PCKGD','MEAT-WHSE','PORK','SPIRITS','NUTRITION',
       'PRODUCE','CHEF SHOPPE','RESTAURANT','SALAD BAR','DELI', 'DAIRY DELI','DELI/SNACK BAR',
    'DRUG GM','COSMETICS','PHARMACY SUPPLY','GARDEN CENTER','FLORAL', 'TRAVEL & LEISUR', 
    'POSTAL CENTER', 'PHOTO', 'VIDEO','VIDEO RENTAL','TOYS','HOUSEWARES', 
    'KIOSK-GAS', 'ELECT &PLUMBING','AUTOMOTIVE','GM MERCH EXP','COUP/STR & MFG', 
     'CHARITABLE CONT', 'RX','PROD-WHS SALES', 'HBC','CNTRL/STORE SUP','MISC. TRANS.', 'MISC SALES TRAN',' ']

product['DEPARTMENT'] = pd.Categorical(product['DEPARTMENT'],
                      ordered = True,
                      categories = ordered_department)

print ('The order of the department categories is :', ordered_department)

In [ ]:
department= product.groupby(['DEPARTMENT']).count()
department[department.BRAND>100].BRAND.plot.bar()


In [ ]:
# if we ever want to use 
Unit=product.CURR_SIZE_OF_PRODUCT.str.extract('([A-Z]\w{0,})',expand = False)
Measure=product.CURR_SIZE_OF_PRODUCT.str.extract('(\d+)',expand = False)

In [ ]:
product.COMMODITY_DESC.unique()